In [1]:
from ipywidgets import widgets
import io
import pandas as pd
import base64
import re

In [2]:
uploader = widgets.FileUpload()

In [3]:
inp = widgets.Text(
    value=path if 'path' in vars() else '',
    description='Absolute path to directory with audio files (modify here):',
    placeholder='Please upload a sound selection table file',
    disabled=False,
    style={'description_width': 'initial'},
    layout={'width': '80%'}
)

In [4]:
modify_btn = widgets.Button(
    description='Modify file',
    layout=widgets.Layout(height='auto', width='300px')
)

In [5]:
path_type_radio_select = widgets.Box(
    [
        widgets.Label(value='Make output path compatible with:'),
        widgets.RadioButtons(
            options=[
                'Windows',
                'all other platforms (Mac, Linux)',
            ],
            layout={'width': 'max-content'}
        )
    ]
)

In [6]:
retain_segments_radio_select = widgets.Box(
    [
        widgets.Label(value='Retain extra path segments:'),
        widgets.RadioButtons(
            options=[
                '0  (only keep the filename, default)',
                '1',
                '2'
            ],
            layout={'width': 'max-content'}
        )
    ]
)

In [7]:
link_placeholder = widgets.HTML()

In [8]:
vbox = widgets.VBox([
    widgets.HTML('<h3>1. Use the button below to upload a sound selection table file</h3>'),
    uploader,
    widgets.HTML('<h3>2. Modify the absolute path below</h3>'),
    inp,
    widgets.HTML(
        '''
        <h3>3. How many segments of the old path to retain?</h3>
        <p>This is useful if you have your data organized in subfolders.</p>
        <p>For instance, if you have two paths <code>c:\path_to_data\site_A\some_filename.wav</code> and <code>c:\path_to_data\site_B\some_filename.wav</code>,
         you might want to retain an extra segment of the path (on top of the file name). Retaining a single segment would mean you are modifying only
         <code>c:\path_to_data</code> and <code>site_B\some_filename.wav</code> would be appended untouched.
        '''
    ),
    retain_segments_radio_select,
    widgets.HTML('<h3>4. Make the path compatible with Windows or other platforms</h3>'),
    path_type_radio_select,
    widgets.HTML('<h3>5. Press the button below to modify the file and display a download link</h3>'),
    modify_btn,
    link_placeholder
])

In [9]:
vbox

In [10]:
def on_change(change):
    global df, path, path_seperator
    
    if change['type'] == 'change':
        if uploader.value:
            uploaded_txt_content = uploader.value[list(uploader.value.keys())[0]]['content']
            string_io = io.StringIO(uploaded_txt_content.decode())
            df = pd.read_csv(string_io, sep="\t")
            path_seperator = '\\' if '\\' in df['Begin Path'].iloc[0] else '/'
#             fns = df['Begin Path'].apply(lambda v: v.split('\\').pop())
            path = df['Begin Path'].apply(lambda v: path_seperator.join(v.split(path_seperator)[:-1]))[0]
            inp.value = path

In [11]:
def download_link_html(df, title = "Download modified file"):
    uploaded_fn=list(uploader.value.keys())[0]
    new_fn_chunks = uploaded_fn.split('.')
    new_fn_chunks[-2] += '_modified'
    new_fn = '.'.join(new_fn_chunks)
    
    segments_to_retain = int(retain_segments_radio_select.children[1].value[0])
    path_segments_to_retain = df['Begin Path'].apply(lambda v: path_seperator.join(v.split(path_seperator)[-(segments_to_retain+1):]))

    new_path = inp.value.rstrip('\\').rstrip('/')
    df['Begin Path'] = path_segments_to_retain.apply(lambda fn_or_path: f'{new_path}{path_seperator}{fn_or_path}')
    
    if path_type_radio_select.children[-1].value == 'Windows':
        df['Begin Path'] = df['Begin Path'].apply(lambda fn: re.subn('/', r'\\', fn)[0])
    else:
        df['Begin Path'] = df['Begin Path'].apply(lambda fn: re.subn(r'\\', '/', fn)[0])
    
    csv = df.to_csv(index=False, sep='\t', float_format='%11.9f')
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=new_fn)
    return html

In [12]:
def display_download_link(_):
    vbox.children = vbox.children[:-1]
    vbox.children += (widgets.HTML(download_link_html(df)),)

In [13]:
uploader.observe(on_change)

In [14]:
modify_btn.on_click(display_download_link)